In [30]:
# set dependencies
import os
import csv
import requests
import time
import psycopg2
import pymongo
import json
import pprint
import urllib.request

import pandas as pd
import numpy as np

from flask_pymongo import PyMongo
from splinter import Browser
from bs4 import BeautifulSoup as bs
from flask import Flask, render_template, redirect
from api_keys import nps_key

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows=100

# National Parks Visitor table

In [31]:
# Get data table
tables=pd.read_html('https://www.nps.gov/aboutus/visitation-numbers.htm', index_col=0)
table=pd.DataFrame(tables[1])
table

,Park,Recreational Visits
1,Great Smoky Mountains National Park,12.1 million
2,Yellowstone National Park,"3,8 million"
3,Zion National Park,3.6 million
4,Rocky Mountain National Park,3.3 million
5,Grand Teton National Park,3.3 million
6,Grand Canyon National Park,2.9 million
7,Cuyahoga Valley National Park,2.8 million
8,Acadia National Park,2.7 million
9,Olympic National Park,2.5 million
10,Joshua Tree National Park,2.4 million


# Kaggle National Parks

In [32]:
parks=pd.read_csv('dataFiles/parks.csv')
parks.head()

,Park Code,Park Name,State,Acres,Latitude,Longitude
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21
1,ARCH,Arches National Park,UT,76519,38.68,-109.57
2,BADL,Badlands National Park,SD,242756,43.75,-102.50
3,BIBE,Big Bend National Park,TX,801163,29.25,-103.25
4,BISC,Biscayne National Park,FL,172924,25.65,-80.08


# Kaggle National Parks species

In [33]:
species=pd.read_csv('dataFiles/species.csv',index_col=0,skipinitialspace=True,usecols=[1,2,3,4,5,6,7,8,9,10,11,12])
species.head()

,Category,Order,Family,Scientific Name,Common Names,Record Status,Occurrence,Nativeness,Abundance,Seasonality,Conservation Status
Park Name,,,,,,,,,,,
Acadia National Park,Mammal,Artiodactyla,Cervidae,Alces alces,Moose,Approved,Present,Native,Rare,Resident,NaN
Acadia National Park,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Approved,Present,Native,Abundant,NaN,NaN
Acadia National Park,Mammal,Carnivora,Canidae,Canis latrans,"Coyote, Eastern Coyote",Approved,Present,Not Native,Common,NaN,Species of Concern
Acadia National Park,Mammal,Carnivora,Canidae,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Approved,Not Confirmed,Native,NaN,NaN,Endangered
Acadia National Park,Mammal,Carnivora,Canidae,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Approved,Present,Unknown,Common,Breeder,NaN


# National Park Service API set-up

In [34]:
base_url='https://developer.nps.gov/api/v1'

# API endpoints 
nps_api_list=['/activities','/activities/parks','/alerts','/amenities','/amenities/parksplaces',
              '/amenities/parksvisitorcenters','/articles','/campgrounds','/events','/lessonplans','/newsreleases','/parks',
              '/passportstamplocations','/people','/places','/thingstodo','/topics','/topics/parks','/tours','/visitorcenters',
              '/webcams']

# API Name list
nps_api_names=['activities','activitiesparks','alerts','amenities','amenitiesparksplaces','amenitiesparksvisitorcenters',
               'articles','campgrounds','events','lessonplans','newsreleases','parks','passportstamplocations','people',
               'places','thingstodo','topics','topicsparks','tours','visitorcenters','webcams']

# Merge table and parks dataframes to reduce parks to just top 10, create code list and convert list to lowercase
table_merged=table.merge(parks,'left',left_on='Park',right_on='Park Name')
nps_park_codes= parks['Park Code'].tolist()
top_ten_park_codes=table_merged['Park Code'].tolist()
top_ten_park_codes=[x.lower() for x in top_ten_park_codes]

# Pretty print result
pprint.pformat(top_ten_park_codes,compact=True)

"['grsm', 'yell', 'zion', 'romo', 'grte', 'grca', 'cuva', 'acad', 'olym', 'jotr']"

# Greater Smoky Mountains

In [35]:
# Extract Greater Smoky Mountains Park endpoint data to see what is available
url=f'{base_url}/parks?parkCode=grsm&api_key={nps_key}&limit=100'
response=requests.get(url)
grsm=json.loads(response.content.decode('utf-8'))
grsm_data=dict(grsm['data'][0])
grsm_data

{'id': 'D9819727-18DF-4A84-BDDE-D4F2696DE340',
 'url': 'https://www.nps.gov/grsm/index.htm',
 'fullName': 'Great Smoky Mountains National Park',
 'parkCode': 'grsm',
 'description': "Ridge upon ridge of forest straddles the border between North Carolina and Tennessee in Great Smoky Mountains National Park. World renowned for its diversity of plant and animal life, the beauty of its ancient mountains, and the quality of its remnants of Southern Appalachian mountain culture, this is America's most visited national park.",
 'latitude': '35.60116374',
 'longitude': '-83.50818326',
 'latLong': 'lat:35.60116374, long:-83.50818326',
 'activities': [{'id': '09DF0950-D319-4557-A57E-04CD2F63FF42',
   'name': 'Arts and Culture'},
  {'id': 'C0FFC3D5-0E74-4B6F-B69D-6F95B78906CE', 'name': 'Live Music'},
  {'id': '5F723BAD-7359-48FC-98FA-631592256E35', 'name': 'Auto and ATV'},
  {'id': '0B4A5320-216D-451A-9990-626E1D5ACE28', 'name': 'Scenic Driving'},
  {'id': '13A57703-BB1A-41A2-94B8-53B692EB7238', 

In [36]:
# Extract a couple data items for further investigation
description=grsm_data['description']
grsm_activities=[activity['name'] for activity in grsm_data['activities']]
grsm_activities

['Arts and Culture',
 'Live Music',
 'Auto and ATV',
 'Scenic Driving',
 'Astronomy',
 'Stargazing',
 'Biking',
 'Road Biking',
 'Camping',
 'Backcountry Camping',
 'Car or Front Country Camping',
 'Horse Camping (see also Horse/Stock Use)',
 'Group Camping',
 'RV Camping',
 'Fishing',
 'Freshwater Fishing',
 'Fly Fishing',
 'Food',
 'Picnicking',
 'Guided Tours',
 'Self-Guided Tours - Auto',
 'Hands-On',
 'Citizen Science',
 'Hiking',
 'Backcountry Hiking',
 'Front-Country Hiking',
 'Horse Trekking',
 'Horse Camping (see also camping)',
 'Horseback Riding',
 'Junior Ranger Program',
 'Wildlife Watching',
 'Birdwatching',
 'Park Film',
 'Museum Exhibits',
 'Shopping',
 'Bookstore and Park Store']

In [37]:
# Transform GRSM request into dictionary to view keys and data types more easily
df=pd.DataFrame.from_dict(grsm_data,orient='index')
df

,0
id,D9819727-18DF-4A84-BDDE-D4F2696DE340
url,https://www.nps.gov/grsm/index.htm
fullName,Great Smoky Mountains National Park
parkCode,grsm
description,Ridge upon ridge of forest straddles the borde...
latitude,35.60116374
longitude,-83.50818326
latLong,"lat:35.60116374, long:-83.50818326"
activities,[{'id': '09DF0950-D319-4557-A57E-04CD2F63FF42'...
topics,[]


In [38]:
# Extract operating hours data to see if anything could be used
grsm_op_hours=grsm_data['operatingHours']
grsm_op_hours

[{'exceptions': [],
  'description': 'Primary roads are open 24 hours a day, seven days a week, year-round, weather permitting. This includes Newfound Gap Road (US-441), Little River Road, and Laurel Creek Road. Secondary roads are closed seasonally.',
  'standardHours': {'wednesday': 'All Day',
   'monday': 'All Day',
   'thursday': 'All Day',
   'sunday': 'All Day',
   'tuesday': 'All Day',
   'friday': 'All Day',
   'saturday': 'All Day'},
  'name': 'Great Smoky Mountains National Park'},
 {'exceptions': [],
  'description': 'Cades Cove Loop Road is open to motor vehicles from sunrise until sunset daily. Pedestrians and bicyclists may access the road 24 hours a day.',
  'standardHours': {'wednesday': 'Sunrise to Sunset',
   'monday': 'Sunrise to Sunset',
   'thursday': 'Sunrise to Sunset',
   'sunday': 'Sunrise to Sunset',
   'tuesday': 'Sunrise to Sunset',
   'friday': 'Sunrise to Sunset',
   'saturday': 'Sunrise to Sunset'},
  'name': 'Cades Cove'}]

In [39]:
# Transform activities result to a count 
len(grsm_activities)

36

# Yellowstone National Park

In [40]:
# Extract YELL /park endpoint data and view results
url=f'{base_url}/parks?parkCode=yell&api_key={nps_key}&limit=100'
response=requests.get(url)
yell=json.loads(response.content.decode('utf-8'))
yell_data=dict(yell['data'][0])
yell_data

{'id': 'F58C6D24-8D10-4573-9826-65D42B8B83AD',
 'url': 'https://www.nps.gov/yell/index.htm',
 'fullName': 'Yellowstone National Park',
 'parkCode': 'yell',
 'description': 'On March 1, 1872, Yellowstone became the first national park for all to enjoy the unique hydrothermal wonders. Today, millions of people come here each year to camp, hike, and enjoy the majesty of the park.',
 'latitude': '44.59824417',
 'longitude': '-110.5471695',
 'latLong': 'lat:44.59824417, long:-110.5471695',
 'activities': [{'id': '09DF0950-D319-4557-A57E-04CD2F63FF42',
   'name': 'Arts and Culture'},
  {'id': '5F723BAD-7359-48FC-98FA-631592256E35', 'name': 'Auto and ATV'},
  {'id': '0B4A5320-216D-451A-9990-626E1D5ACE28', 'name': 'Scenic Driving'},
  {'id': '13A57703-BB1A-41A2-94B8-53B692EB7238', 'name': 'Astronomy'},
  {'id': '7CE6E935-F839-4FEC-A63E-052B1DEF39D2', 'name': 'Biking'},
  {'id': '299CB934-88DC-474F-A33D-E43E29A149C2', 'name': 'Mountain Biking'},
  {'id': '8D778629-F603-4C50-A121-6F4BB2FE2C4B', 

# Yellowstone

In [41]:
# Extract a elements, transform activity data into a list then count for comparison
yell_description=yell_data['description']
len_test=len([activity['name'] for activity in yell_data['activities']])
len_test

53

In [42]:
# Extract operating hours from query result for comparison
yell_op_hours=yell_data['operatingHours']
yell_op_hours

[{'exceptions': [],
  'description': 'Yellowstone is open daily, year-round, although activities and services are limited at night and certain times of year. The park has five entrance stations, but not all entrance stations are open year-round. Make sure to carefully read about access at each station at different times of year. And remember, all dates are weather-dependent.\n\nPlease note that camping is possible only in designated campgrounds.',
  'standardHours': {'wednesday': 'All Day',
   'monday': 'All Day',
   'thursday': 'All Day',
   'sunday': 'All Day',
   'tuesday': 'All Day',
   'friday': 'All Day',
   'saturday': 'All Day'},
  'name': 'All Park Hours'},
 {'exceptions': [{'exceptionHours': {'wednesday': 'Closed',
     'monday': 'Closed',
     'thursday': 'Closed',
     'sunday': 'Closed',
     'tuesday': 'Closed',
     'friday': 'Closed',
     'saturday': 'Closed'},
    'startDate': '2022-03-16',
    'name': 'Spring Shoulder Season',
    'endDate': '2022-04-16'},
   {'excep

# Top 10 parks combined search

In [43]:
# After reviewing multiple endpoints in similar fasion the most relevent item was activities
# Query the top 10 visited parks in 2020, extract activity data, transform into counts for each park and create list.
activity_count=[]
for i in top_ten_park_codes:
    url=f'{base_url}/parks?parkCode={i}&api_key={nps_key}'
    response=requests.get(url)
    top_ten=json.loads(response.content.decode('utf-8'))
    top_ten_a=dict(top_ten['data'][0])
    park_act=len([activity['name'] for activity in top_ten_a['activities']])
    activity_count.append(park_act)

In [44]:
# View results
activity_count

[36, 53, 22, 34, 53, 33, 31, 46, 54, 27]

In [46]:
# Convert list to dataframe, merge with table dataframe, and clean combined dataframe
count_df=pd.DataFrame(activity_count,columns=['Number of Activities'])
table=pd.DataFrame(tables[1]).reset_index()
park_activity=table.merge(count_df,'left',left_index=True,right_index=True)
park_activity=park_activity.rename(columns={'index':'Rank'})
park_activity=park_activity.set_index('Rank')
park_activity

,Park,Recreational Visits,Number of Activities
Rank,,,
1,Great Smoky Mountains National Park,12.1 million,36
2,Yellowstone National Park,"3,8 million",53
3,Zion National Park,3.6 million,22
4,Rocky Mountain National Park,3.3 million,34
5,Grand Teton National Park,3.3 million,53
6,Grand Canyon National Park,2.9 million,33
7,Cuyahoga Valley National Park,2.8 million,31
8,Acadia National Park,2.7 million,46
9,Olympic National Park,2.5 million,54
